In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split

from sklearn.model_selection import GridSearchCV
from sklearn.utils import resample
from sklearn.compose import ColumnTransformer
from sklearn.decomposition import PCA
import seaborn as sns
import numpy as np
import pickle
from sklearn.ensemble import RandomForestRegressor

from sklearn.linear_model import LinearRegression

import matplotlib.pyplot as plt
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.decomposition import FactorAnalysis
from factor_analyzer import FactorAnalyzer

from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import roc_auc_score

%matplotlib inline

from sklearn.preprocessing import OneHotEncoder
import lightgbm as lgb
from lightgbm import LGBMClassifier 

from sklearn.utils import class_weight
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from sklearn.decomposition import TruncatedSVD

import plotly.express as px
import plotly.graph_objs as go
import statsmodels.formula.api as smf
from plotly.graph_objects import Layout

In [2]:
df = pd.read_csv("mergedData.csv")

In [3]:
df = df.drop("Unnamed: 0", axis = 1)

In [4]:
df.columns

Index(['Abbreviation', 'DriverNumber', 'TeamName', 'Position', 'GridPosition',
       'Status', 'Points', 'RaceCountry', 'Year', 'AgeAtGP', 'BestQualiTime',
       'FLap', 'AvgLapTime', 'SDLapTime', 'AvgSplitTime', 'AvgPitTime',
       'PitstopNo', 'HARD', 'INTERMEDIATE', 'MEDIUM', 'SOFT', 'WET', 'Engine',
       'Rain', 'Driver', 'AverageSpeed', 'MaxSpeed', 'AverageRPM', 'MaxRPM',
       'AverageThrottle', 'MaxThrottlePct', 'Brake', 'raceID', 'CircuitType'],
      dtype='object')

In [5]:
df = df.drop(["DriverNumber", "Driver", "Points"], axis = 1)
# df = df.drop(["DriverNumber", "Driver", "Position"], axis = 1)
aux = df

In [6]:
# create new columns based on Status values
df['carIssue'] = (df['Status'] == 'carIssue').astype(int)
df['driverIssue'] = (df['Status'] == 'driverIssue').astype(int)
df = df.drop('Status', axis=1)

In [8]:
df = df.drop(["AverageRPM"], axis = 1)
df = df.drop(["AvgLapTime"], axis = 1)
df = df.drop(["AvgSplitTime"], axis = 1)

In [9]:
df["RaceCountry"] = df["RaceCountry"].str.replace(" ", "")
df["TeamName"] = df["TeamName"].str.replace(" ", "")
df["Engine"] = df["Engine"].str.replace(" ", "")

In [10]:
#df.to_csv(r'df_points.csv', index=True, header=True)
#df.to_csv(r'df_position.csv', index=True, header=True)
#df.to_csv(r'plotdata_points.csv', index=True, header=True)
#df.to_csv(r'plotdata_position.csv', index=True, header=True)

In [11]:

# get a list of all categorical variables except "Abbreviation" and "raceID"
cat_vars = df.select_dtypes(include=['object']).columns.tolist()
cat_vars.remove("Abbreviation")
cat_vars.remove("RaceCountry")

# create dummy variables for all categorical variables except "Abbreviation" and "raceID"
dummies = pd.get_dummies(df[cat_vars], drop_first=True)

# drop the original categorical variables from the DataFrame
df = df.drop(cat_vars, axis=1)

# concatenate the dummy variables with the remaining variables in the DataFrame
df = pd.concat([df, dummies], axis=1)


In [16]:
df["Year"].value_counts()

2022.0    424
2019.0    414
2021.0    401
2020.0    308
Name: Year, dtype: int64

In [17]:
# Select only numeric variables
numeric_cols = df.select_dtypes(include=['float64', 'int64']).columns.tolist()

# Compute the correlation matrix for numeric variables
corr_matrix = df[numeric_cols].corr()

# Select variables with correlation above 0.7 with "Position"
corr_vars = corr_matrix[(corr_matrix['Position'] > 0.5) & (corr_matrix['Position'] < 1.0)].index.tolist()

# Print the correlated variables
print(corr_vars)

['GridPosition']


In [18]:
corr_matrix

,Position,GridPosition,Year,AgeAtGP,BestQualiTime,FLap,SDLapTime,AvgPitTime,PitstopNo,HARD,...,WET,AverageSpeed,MaxSpeed,MaxRPM,AverageThrottle,MaxThrottlePct,Brake,raceID,carIssue,driverIssue
Position,1.000000,0.627888,-0.012545,-0.054946,0.063725,-0.254654,-0.084467,-0.067249,-0.093845,-0.074772,...,0.009832,-0.338325,0.003477,-0.000516,0.240732,0.263711,0.425201,-0.011875,0.499157,0.187402
GridPosition,0.627888,1.000000,0.004768,-0.043347,0.096416,-0.255264,-0.032930,-0.016867,-0.017870,-0.010745,...,0.002519,-0.063584,0.077044,0.074328,0.049436,0.024112,0.128357,0.005169,0.060852,0.064120
Year,-0.012545,0.004768,1.000000,0.049934,0.037337,0.002073,0.201683,0.137231,0.131212,0.058892,...,-0.008292,-0.358282,-0.177155,0.083694,-0.171381,-0.025770,0.133608,0.966501,-0.000671,-0.008879
AgeAtGP,-0.054946,-0.043347,0.049934,1.000000,-0.005432,0.021246,0.011240,0.023946,0.004781,-0.000873,...,-0.010955,0.001599,-0.003354,0.050426,0.000097,-0.025138,-0.009368,0.055430,-0.004221,-0.038683
BestQualiTime,0.063725,0.096416,0.037337,-0.005432,1.000000,-0.024219,-0.145525,0.085063,-0.120084,-0.127630,...,0.153041,-0.088104,0.077776,0.045474,0.030439,0.045238,0.066518,0.060311,0.025227,0.061837
FLap,-0.254654,-0.255264,0.002073,0.021246,-0.024219,1.000000,0.025585,-0.016594,0.131131,-0.008590,...,-0.001471,0.050216,-0.012277,0.003043,-0.011706,-0.020203,-0.057344,0.001833,-0.073601,-0.009809
SDLapTime,-0.084467,-0.032930,0.201683,0.011240,-0.145525,0.025585,1.000000,0.227640,0.429050,-0.085564,...,0.072518,-0.154892,-0.018419,0.059878,-0.165944,-0.037257,0.007745,0.166882,-0.021592,-0.005386
AvgPitTime,-0.067249,-0.016867,0.137231,0.023946,0.085063,-0.016594,0.227640,1.000000,0.274141,-0.025267,...,-0.002306,-0.263082,-0.026570,0.050139,-0.336918,-0.051029,-0.116323,0.120346,-0.013116,-0.024164
PitstopNo,-0.093845,-0.017870,0.131212,0.004781,-0.120084,0.131131,0.429050,0.274141,1.000000,-0.079034,...,0.134203,0.020333,0.022582,0.113893,-0.377962,-0.236534,-0.250675,0.120589,-0.196837,-0.077487
HARD,-0.074772,-0.010745,0.058892,-0.000873,-0.127630,-0.008590,-0.085564,-0.025267,-0.079034,1.000000,...,-0.199766,0.182815,0.001504,0.014888,-0.127590,-0.200048,-0.203713,0.053751,-0.195583,-0.045041


## MERF

In [19]:
from merf import MERF

In [20]:
df[["Year", "raceID"]]

,Year,raceID
0,2019.0,1
1,2019.0,1
2,2019.0,1
3,2019.0,1
4,2019.0,1
...,...,...
1542,2022.0,80
1543,2022.0,80
1544,2022.0,80
1545,2022.0,80


In [21]:
# Split the data into two datasets based on the Year column
df_before = df[df['Year'].isin([2019, 2020])]
df_after = df[df['Year'].isin([2021, 2022])]

## Before model

In [22]:
df_before["raceID"]

0       1
1       1
2       1
3       1
4       1
       ..
717    37
718    37
719    37
720    37
721    37
Name: raceID, Length: 722, dtype: int64

In [23]:
# Split the data into train and test sets based on the raceID column
train_df = df_before[(df_before['raceID'] >= 1) & (df_before['raceID'] <= 29)]
test_df = df_before[(df_before['raceID'] >= 30) & (df_before['raceID'] <= 37)]

In [24]:
X_train = train_df.drop('Position', axis=1)
Y_train = train_df['Position']
X_test = test_df.drop('Position', axis=1)
Y_test = test_df['Position']

In [25]:
X_train = X_train.drop('RaceCountry', axis=1)
X_train = X_train.drop('Year', axis=1)
X_test = X_test.drop('RaceCountry', axis=1)
X_test = X_test.drop('Year', axis=1)

In [26]:
x_train = X_train.drop(['Abbreviation', 'raceID'], axis = 1)
Z_train = X_train[['raceID']]
clusters_train = X_train['raceID']

In [27]:
from lightgbm import LGBMRegressor
lgbm = LGBMRegressor(max_depth = 10, learning_rate = 0.05, n_estimators = 1000)

In [28]:
mrf = MERF(lgbm, max_iterations=30)
#mrf = MERF(fixed_effects_model = LinearRegression(), max_iterations=30)
mrf.fit(x_train, Z_train, clusters_train, Y_train)


INFO     [merf.py:307] Training GLL is -1814.2977514988797 at iteration 1.
INFO     [merf.py:307] Training GLL is -3111.3117441397603 at iteration 2.
INFO     [merf.py:307] Training GLL is -3262.0117527020284 at iteration 3.
INFO     [merf.py:307] Training GLL is -3370.730770892507 at iteration 4.
INFO     [merf.py:307] Training GLL is -3300.4949597791015 at iteration 5.
INFO     [merf.py:307] Training GLL is -3264.4628998285457 at iteration 6.
INFO     [merf.py:307] Training GLL is -3361.562173619673 at iteration 7.
INFO     [merf.py:307] Training GLL is -3376.0196199996117 at iteration 8.
INFO     [merf.py:307] Training GLL is -3377.2413756351207 at iteration 9.
INFO     [merf.py:307] Training GLL is -3259.731444384213 at iteration 10.
INFO     [merf.py:307] Training GLL is -3333.4668419882823 at iteration 11.
INFO     [merf.py:307] Training GLL is -3391.2086924245114 at iteration 12.
INFO     [merf.py:307] Training GLL is -3410.028867853237 at iteration 13.
INFO     [merf.py:307] Tr

In [29]:
train_preds = mrf.predict(x_train, Z_train, clusters_train)

In [30]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

mse = mean_squared_error(Y_train, train_preds)
rmse = np.sqrt(mse)
mae = mean_absolute_error(Y_train, train_preds)
r2 = r2_score(Y_train, train_preds)

# Print the performance metrics
print("Test MSE:", mse)
print("Test RMSE:", rmse)
print("Test MAE:", mae)
print("Test R-squared:", r2)


Test MSE: 0.0017845513954319317
Test RMSE: 0.04224395099220635
Test MAE: 0.02954712143906553
Test R-squared: 0.9999452055224932


# Test dataset

In [31]:
x_test = X_test.drop(['Abbreviation', 'raceID'], axis = 1)
Z_test = X_test[['raceID']]
clusters_test = X_test['raceID']

In [32]:
x_test

,GridPosition,AgeAtGP,BestQualiTime,FLap,SDLapTime,AvgPitTime,PitstopNo,HARD,INTERMEDIATE,MEDIUM,...,TeamName_RacingPoint,TeamName_RedBullRacing,TeamName_Renault,TeamName_ToroRosso,TeamName_Williams,Engine_Honda,Engine_Mercedes,Engine_RedBull,Engine_Renault,CircuitType_street
568,9.0,24.0,93.000,0.0,4.414441,29.5895,2.0,24.0,0.0,11.0,...,0,0,0,0,0,0,0,0,0,1
569,4.0,30.0,92.317,0.0,3.598099,29.6020,1.0,33.0,0.0,0.0,...,1,0,0,0,0,0,1,0,0,1
570,10.0,22.0,93.239,0.0,3.308658,30.2610,1.0,25.0,0.0,28.0,...,0,0,0,0,0,0,0,0,0,1
571,18.0,28.0,94.681,0.0,3.643243,30.1880,1.0,33.0,0.0,19.0,...,0,0,0,0,0,0,0,0,0,1
572,15.0,24.0,93.008,0.0,3.737778,30.1610,2.0,26.0,0.0,26.0,...,0,1,0,0,0,1,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
717,18.0,25.0,98.443,0.0,7.947119,21.8705,2.0,24.0,0.0,30.0,...,0,0,0,0,1,0,1,0,0,0
718,16.0,21.0,98.045,0.0,8.040811,22.0850,1.0,44.0,0.0,10.0,...,0,0,0,0,1,0,1,0,0,0
719,15.0,41.0,97.555,0.0,8.160983,22.0120,1.0,44.0,0.0,10.0,...,0,0,0,0,0,0,0,0,0,0
720,2.0,31.0,95.271,0.0,3.679513,21.5870,1.0,45.0,0.0,10.0,...,0,0,0,0,0,0,1,0,0,0


In [33]:
y_pred = mrf.predict(x_test, Z_test, clusters_test)

In [34]:
mse = mean_squared_error(Y_test, y_pred)
rmse = np.sqrt(mse)
mae = mean_absolute_error(Y_test, y_pred)
r2 = r2_score(Y_test, y_pred)

# Print the performance metrics
print("Test MSE:", mse)
print("Test RMSE:", rmse)
print("Test MAE:", mae)
print("Test R-squared:", r2)


Test MSE: 10.651847412729614
Test RMSE: 3.263716809517887
Test MAE: 2.482383290847572
Test R-squared: 0.6592162931861992


## After model

In [35]:
df_after["raceID"]

722     38
723     38
724     38
725     38
726     38
        ..
1542    80
1543    80
1544    80
1545    80
1546    80
Name: raceID, Length: 825, dtype: int64

In [36]:
df_after

,Abbreviation,Position,GridPosition,RaceCountry,Year,AgeAtGP,BestQualiTime,FLap,SDLapTime,AvgPitTime,...,TeamName_RacingPoint,TeamName_RedBullRacing,TeamName_Renault,TeamName_ToroRosso,TeamName_Williams,Engine_Honda,Engine_Mercedes,Engine_RedBull,Engine_Renault,CircuitType_street
722,GAS,17.0,5.0,Bahrain,2021.0,24.0,89.809,0.0,9.870618,29.212667,...,0,0,0,0,0,0,0,0,0,0
723,PER,5.0,20.0,Bahrain,2021.0,31.0,90.659,0.0,9.238201,24.096333,...,0,1,0,0,0,1,0,0,0,0
724,ALO,19.0,9.0,Bahrain,2021.0,39.0,90.249,0.0,9.050427,24.574000,...,0,0,0,0,0,0,0,0,1,0
725,LEC,6.0,4.0,Bahrain,2021.0,23.0,89.678,0.0,7.326363,24.550500,...,0,0,0,0,0,0,0,0,0,0
726,STR,10.0,10.0,Bahrain,2021.0,22.0,90.601,0.0,9.904268,25.204500,...,0,0,0,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1542,VET,10.0,9.0,AbuDhabi,2022.0,35.0,84.961,0.0,3.057131,23.186000,...,0,0,0,0,0,0,1,0,0,0
1543,SAI,4.0,4.0,AbuDhabi,2022.0,28.0,84.242,0.0,3.799668,21.752500,...,0,0,0,0,0,0,0,0,0,0
1544,LAT,19.0,20.0,AbuDhabi,2022.0,27.0,86.054,0.0,6.817590,23.798500,...,0,0,0,0,1,0,1,0,0,0
1545,RUS,5.0,6.0,AbuDhabi,2022.0,23.0,84.511,0.0,4.489247,26.087000,...,0,0,0,0,0,0,1,0,0,0


In [37]:
# Split the data into train and test sets based on the raceID column
train_df = df_after[(df_after['raceID'] >= 38) & (df_after['raceID'] <= 72)]
test_df = df_after[(df_after['raceID'] >= 73) & (df_after['raceID'] <= 80)]

In [38]:
X_train = train_df.drop('Position', axis=1)
Y_train = train_df['Position']
X_test = test_df.drop('Position', axis=1)
Y_test = test_df['Position']

In [39]:
X_train = X_train.drop('RaceCountry', axis=1)
X_train = X_train.drop('Year', axis=1)
X_test = X_test.drop('RaceCountry', axis=1)
X_test = X_test.drop('Year', axis=1)

In [40]:
x_train = X_train.drop(['Abbreviation', 'raceID'], axis = 1)
Z_train = X_train[['raceID']]
clusters_train = X_train['raceID']

In [41]:

#mrf = MERF(fixed_effects_model = LinearRegression(), max_iterations=30)
mrf.fit(x_train, Z_train, clusters_train, Y_train)


INFO     [merf.py:307] Training GLL is -2298.994457420132 at iteration 1.
INFO     [merf.py:307] Training GLL is -3694.9260565979284 at iteration 2.
INFO     [merf.py:307] Training GLL is -3740.356689626113 at iteration 3.
INFO     [merf.py:307] Training GLL is -3663.7045493709566 at iteration 4.
INFO     [merf.py:307] Training GLL is -3804.452324511743 at iteration 5.
INFO     [merf.py:307] Training GLL is -3635.948743023698 at iteration 6.
INFO     [merf.py:307] Training GLL is -3793.622996571447 at iteration 7.
INFO     [merf.py:307] Training GLL is -3617.753548993378 at iteration 8.
INFO     [merf.py:307] Training GLL is -3791.6654470411268 at iteration 9.
INFO     [merf.py:307] Training GLL is -3738.580460283372 at iteration 10.
INFO     [merf.py:307] Training GLL is -3820.277162151804 at iteration 11.
INFO     [merf.py:307] Training GLL is -3918.1384085007917 at iteration 12.
INFO     [merf.py:307] Training GLL is -3825.8194505949473 at iteration 13.
INFO     [merf.py:307] Traini

In [42]:
train_preds = mrf.predict(x_train, Z_train, clusters_train)

In [43]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

mse = mean_squared_error(Y_train, train_preds)
rmse = np.sqrt(mse)
mae = mean_absolute_error(Y_train, train_preds)
r2 = r2_score(Y_train, train_preds)

# Print the performance metrics
print("Test MSE:", mse)
print("Test RMSE:", rmse)
print("Test MAE:", mae)
print("Test R-squared:", r2)

Test MSE: 0.002784377358501824
Test RMSE: 0.052767199646199
Test MAE: 0.037812576651521206
Test R-squared: 0.9999118976241722


# Test Dataset

In [44]:
x_test = X_test.drop(['Abbreviation', 'raceID'], axis = 1)
Z_test = X_test[['raceID']]
clusters_test = X_test['raceID']

In [45]:
y_pred = mrf.predict(x_test, Z_test, clusters_test)

In [46]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

mse = mean_squared_error(Y_test, y_pred)
rmse = np.sqrt(mse)
mae = mean_absolute_error(Y_test, y_pred)
r2 = r2_score(Y_test, y_pred)

# Print the performance metrics
print("Test MSE:", mse)
print("Test RMSE:", rmse)
print("Test MAE:", mae)
print("Test R-squared:", r2)


Test MSE: 10.28143895532415
Test RMSE: 3.2064682994416382
Test MAE: 2.4254420680307494
Test R-squared: 0.6741985648239796
